# Predicting Crime in US Communites
## Team member: Bibata Rabba Idi, Fatima Javid, JianHui (Jake) Li 

In [14]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [60]:
# Load data
df = pd.read_csv('crimedata.csv')
df.head()

,communityName,state,countyCode,communityCode,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
0,BerkeleyHeightstownship,NJ,39.0,5320.0,11980,3.10,1.37,91.78,6.50,1.88,...,14.0,114.85,138.0,1132.08,16.0,131.26,2.0,16.41,41.02,1394.59
1,Marpletownship,PA,45.0,47616.0,23123,2.82,0.80,95.57,3.44,0.85,...,57.0,242.37,376.0,1598.78,26.0,110.55,1.0,4.25,127.56,1955.95
2,Tigardcity,OR,NaN,NaN,29344,2.43,0.74,94.33,3.43,2.35,...,274.0,758.14,1797.0,4972.19,136.0,376.30,22.0,60.87,218.59,6167.51
3,Gloversvillecity,NY,35.0,29443.0,16656,2.40,1.70,97.35,0.50,0.70,...,225.0,1301.78,716.0,4142.56,47.0,271.93,NaN,NaN,306.64,NaN
4,Bemidjicity,MN,7.0,5068.0,11245,2.76,0.53,89.16,1.17,0.52,...,91.0,728.93,1060.0,8490.87,91.0,728.93,5.0,40.05,NaN,9988.79


#### Since this dataset is very similar to this [UCI Machine Learning repo](https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime), you can find most attribute information from there. We will define the attributes that's not included in the repo.
communityName:

state:

countyCode:

communityCode:

population:

householdsize:

racepctblack:

racePctWhite:

racePctAsian:

racePctHisp:

agePct12t21:

agePct12t29:

agePct16t24:

agePct65up:

numbUrban:

pctUrban:

medIncome:

pctWWage:

pctWFarmSelf:

pctWInvInc:

pctWSocSec:

pctWPubAsst:

pctWRetire:

medFamInc:

perCapInc:

whitePerCap:

blackPerCap:

indianPerCap:

AsianPerCap:

OtherPerCap:

HispPerCap:

NumUnderPov:

PctPopUnderPov:

PctLess9thGrade:

PctNotHSGrad:

PctBSorMore:

PctUnemployed:

PctEmploy:

PctEmplManu:

PctEmplProfServ:

PctOccupManu:

PctOccupMgmtProf:

MalePctDivorce:

MalePctNevMarr:

FemalePctDiv:

TotalPctDiv:

PersPerFam:

PctFam2Par:

PctKids2Par:

PctYoungKids2Par:

PctTeen2Par:

PctWorkMomYoungKids:

PctWorkMom:

NumKidsBornNeverMar:

PctKidsBornNeverMar:

NumImmig:

PctImmigRecent:

PctImmigRec5:

PctImmigRec8:

PctImmigRec10:

PctRecentImmig:

PctRecImmig5:

PctRecImmig8:

PctRecImmig10:

PctSpeakEnglOnly:

PctNotSpeakEnglWell:

PctLargHouseFam:

PctLargHouseOccup:

PersPerOccupHous:

PersPerOwnOccHous:

PersPerRentOccHous:

PctPersOwnOccup:

PctPersDenseHous:

PctHousLess3BR:

MedNumBR:

HousVacant:

PctHousOccup:

PctHousOwnOcc:

PctVacantBoarded:

PctVacMore6Mos:

MedYrHousBuilt:

PctHousNoPhone:

PctWOFullPlumb:

OwnOccLowQuart:

OwnOccMedVal:

OwnOccHiQuart:

OwnOccQrange:

RentLowQ:

RentMedian:

RentHighQ:

RentQrange:

MedRent:

MedRentPctHousInc:

MedOwnCostPctInc:

MedOwnCostPctIncNoMtg:

NumInShelters:

NumStreet:

PctForeignBorn:

PctBornSameState:

PctSameHouse85:

PctSameCity85:

PctSameState85:

LemasSwornFT:

LemasSwFTPerPop:

LemasSwFTFieldOps:

LemasSwFTFieldPerPop:

LemasTotalReq:

LemasTotReqPerPop:

PolicReqPerOffic:

PolicPerPop:

RacialMatchCommPol:

PctPolicWhite:

PctPolicBlack:

PctPolicHisp:

PctPolicAsian:

PctPolicMinor:

OfficAssgnDrugUnits:

NumKindsDrugsSeiz:

PolicAveOTWorked:

LandArea:

PopDens:

PctUsePubTrans:

PolicCars:

PolicOperBudg:

LemasPctPolicOnPatr:

LemasGangUnitDeploy:

LemasPctOfficDrugUn:

PolicBudgPerPop:

murders:

murdPerPop:

rapes:

rapesPerPop:

robberies:

robbbPerPop:

assaults:

assaultPerPop:

burglaries:

burglPerPop:

larcenies:

larcPerPop:

autoTheft:

autoTheftPerPop:

arsons:

arsonsPerPop:

ViolentCrimesPerPop:

nonViolPerPop:


In [65]:
# Shape of dataframe
df.shape

(2215, 146)

In [67]:
# Number of nulls from each column
df.isnull().sum()

communityName             0
state                     0
countyCode             1221
communityCode          1224
population                0
                       ... 
autoTheftPerPop           3
arsons                   91
arsonsPerPop             91
ViolentCrimesPerPop     221
nonViolPerPop            97
Length: 146, dtype: int64

In [66]:
# Make a list of columns that contain nulls
df.columns[df.isna().any()].tolist()

['countyCode',
 'communityCode',
 'OtherPerCap',
 'LemasSwornFT',
 'LemasSwFTPerPop',
 'LemasSwFTFieldOps',
 'LemasSwFTFieldPerPop',
 'LemasTotalReq',
 'LemasTotReqPerPop',
 'PolicReqPerOffic',
 'PolicPerPop',
 'RacialMatchCommPol',
 'PctPolicWhite',
 'PctPolicBlack',
 'PctPolicHisp',
 'PctPolicAsian',
 'PctPolicMinor',
 'OfficAssgnDrugUnits',
 'NumKindsDrugsSeiz',
 'PolicAveOTWorked',
 'PolicCars',
 'PolicOperBudg',
 'LemasPctPolicOnPatr',
 'LemasGangUnitDeploy',
 'PolicBudgPerPop',
 'rapes',
 'rapesPerPop',
 'robberies',
 'robbbPerPop',
 'assaults',
 'assaultPerPop',
 'burglaries',
 'burglPerPop',
 'larcenies',
 'larcPerPop',
 'autoTheft',
 'autoTheftPerPop',
 'arsons',
 'arsonsPerPop',
 'ViolentCrimesPerPop',
 'nonViolPerPop']

In [68]:
# Check for duplicates
df.duplicated().sum()

0

In [17]:
df.corr()

/tmp/ipykernel_764/1886530979.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,countyCode,communityCode,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
countyCode,1.000000,0.124997,0.080867,-0.032992,0.219294,-0.173580,-0.085545,-0.088787,0.146577,0.145099,...,0.070362,0.032133,0.105864,0.176043,0.036016,-0.014450,0.068501,0.105851,0.084384,0.139461
communityCode,0.124997,1.000000,-0.034680,0.002698,-0.013897,0.014749,0.033570,0.000687,0.007493,-0.023551,...,-0.055203,-0.041514,-0.046555,-0.022198,-0.049772,-0.011705,-0.040293,-0.027595,-0.089463,-0.035875
population,0.080867,-0.034680,1.000000,-0.018841,0.135641,-0.184685,0.088360,0.094048,-0.007624,0.046247,...,0.970569,0.119094,0.966002,0.066389,0.980754,0.244902,0.903354,0.158558,0.212354,0.123401
householdsize,-0.032992,0.002698,-0.018841,1.000000,-0.047455,-0.230117,0.186779,0.485178,0.491481,0.384322,...,-0.034034,-0.102347,-0.048819,-0.237922,-0.011737,0.022456,-0.013160,-0.015553,-0.020110,-0.193963
racepctblack,0.219294,-0.013897,0.135641,-0.047455,1.000000,-0.820605,-0.089300,-0.063911,0.097053,0.119994,...,0.210725,0.548555,0.183407,0.362778,0.158705,0.409258,0.155844,0.284539,0.628368,0.484853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
autoTheftPerPop,-0.014450,-0.011705,0.244902,0.022456,0.409258,-0.557296,0.183556,0.348928,-0.058835,0.069275,...,0.329374,0.540168,0.303032,0.362927,0.307152,1.000000,0.261050,0.353595,0.636484,0.596899
arsons,0.068501,-0.040293,0.903354,-0.013160,0.155844,-0.200187,0.063872,0.099251,-0.001511,0.044412,...,0.894951,0.136532,0.882561,0.073662,0.915053,0.261050,1.000000,0.282861,0.232825,0.141942
arsonsPerPop,0.105851,-0.027595,0.158558,-0.015553,0.284539,-0.340851,0.043911,0.153447,0.025917,0.071776,...,0.212846,0.417039,0.201004,0.307609,0.168580,0.353595,0.282861,1.000000,0.416719,0.411457
ViolentCrimesPerPop,0.084384,-0.089463,0.212354,-0.020110,0.628368,-0.676849,0.031949,0.253596,0.023535,0.105909,...,0.316505,0.698553,0.295022,0.509410,0.244926,0.636484,0.232825,0.416719,1.000000,0.675374
